In [136]:
!pip install langchain langchain_openai faiss-cpu --upgrade --quiet

<a href = "https://colab.research.google.com/github/qxr777/llm-application-code/blob/master/retrieval_augmented_generation/rag-helloworld.ipynb"
target = "_parent"> Open In Colab </a>

In [137]:
# 三选一 加载OPENAI_API_KEY

# 本地运行，包含.env文件
from dotenv import load_dotenv
load_dotenv(override=True)

# 本地运行，不包含.env文件
# import os
# os.environ['OPENAI_API_KEY'] = 'sk-123456789'

# google colab运行，需要配置secret key
# import os
# from google.colab import userdata
# 
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

True

In [138]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [139]:
# 创建LLM
model = 'gpt-4o-mini'
llm = ChatOpenAI(model=model)

In [140]:
# 创建Prompt
prompt = ChatPromptTemplate.from_template('基于上下文：{context}\n回答：{input}')

In [141]:
# 创建输出解析器
output_parser = StrOutputParser()

In [142]:
# 模拟文档
docs = [Document(page_content="课程代码 01041110	课程名称 企业级应用软件设计与开发	课程大类 专业学位硕士类课程	班级 23硕士合班	学生数 165	周次 2-14	时间 【星期四】9,10,11节	上课地点 马区西-东教学楼-133(西东教-合一)	")]

In [143]:
# 文档嵌入
splits = RecursiveCharacterTextSplitter().split_documents(docs)
vector_store = FAISS.from_documents(splits, OpenAIEmbeddings())
retriever = vector_store.as_retriever()

In [144]:
# 创建Chain
chain_no_context = RunnablePassthrough() | llm | output_parser
chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt | llm | output_parser
)

In [145]:
# 调用Chain
question = '企业级应用软件设计与开发的上课地点在哪里？'

# 调用没有上下文的chain
response_no_context = chain_no_context.invoke(question)
print("没有上下文的回答：")
print(response_no_context)
print("\n")

# 调用包含上下文的chain
response_with_context = chain.invoke(question)
print("包含上下文的回答：")
print(response_with_context)


没有上下文的回答：
企业级应用软件设计与开发的课程地点通常取决于具体的教育机构或培训中心。如果你是在大学或职业学院学习，课程地点可能是在校园内的特定教室。如果你参加的是在线课程，那么上课地点则是在你自己的设备上。

为了得到准确的信息，建议你查看具体教育机构的官方网站或联系相关的教学部门，以获取最新的上课地点信息。


包含上下文的回答：
企业级应用软件设计与开发的上课地点是马区西-东教学楼-133(西东教-合一)。
